In [15]:
#### Random Forest Classification Problem
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

df = pd.read_csv("Travel.csv")
df.head()
df.info
### Data cleaning
df.isnull().sum()
# age typeofcontract durationofpitch numberoffollowups preferredpropertystar numberoftrips numberofchilderenvisiting monthlyincome
df['Gender'] = df['Gender'].replace('Fe Male','Female')
df['MaritalStatus'] = df['MaritalStatus'].replace('Single','Unmarried')
df.head()
df['Gender'].value_counts()
df['MaritalStatus'].value_counts()
## Imputing NaN value
df.head()
df.Age.fillna(df.Age.median(), inplace=True)
df.TypeofContact.fillna(df.TypeofContact.mode()[0], inplace=True)
df.DurationOfPitch.fillna(df.DurationOfPitch.median(), inplace=True)
df.NumberOfFollowups.fillna(df.NumberOfFollowups.mode()[0], inplace=True)
df.PreferredPropertyStar.fillna(df.PreferredPropertyStar.mode()[0], inplace=True)
df.NumberOfTrips.fillna(df.NumberOfTrips.median(), inplace=True)
df.NumberOfChildrenVisiting.fillna(df.NumberOfChildrenVisiting.mode()[0], inplace=True)
df.MonthlyIncome.fillna(df.MonthlyIncome.median(), inplace=True)

df.head()
df.isnull().sum()
df.drop('CustomerID', inplace=True , axis=1)
df.head()
## FEature Engineering

df['Total Visiting'] = df['NumberOfPersonVisiting'] + df['NumberOfChildrenVisiting']
df.drop(columns=["NumberOfPersonVisiting","NumberOfChildrenVisiting"], axis=1, inplace=True)
df.head()
numerical_features = []

for column in df.columns:
    if pd.api.types.is_numeric_dtype(df[column]):
        numerical_features.append(column)

print("Numerical Features:", numerical_features)
print("The Sum of numerical Features is :", len(numerical_features))
numerical_features.remove("ProdTaken")
print(numerical_features)

categorical_features = []

for column in df.columns:
    if pd.api.types.is_object_dtype(df[column]) or pd.api.types.is_categorical_dtype(df[column]):
        categorical_features.append(column)

print("Categorical Features:", categorical_features)
print("The Sum of numerical Features is :", len(categorical_features))

discrete_features = []
continuous_features = []

for column in df.columns:
    if pd.api.types.is_numeric_dtype(df[column]):
        if pd.api.types.is_integer_dtype(df[column]):
            discrete_features.append(column)
        elif pd.api.types.is_float_dtype(df[column]):
            continuous_features.append(column)

print("Discrete Features:", discrete_features)
print("Count of Discrete Features:", len(discrete_features))

print("Continuous Features:", continuous_features)
print("Count of Continuous Features:", len(continuous_features))

## TRain TEst Splitt and Model Trainig
X = df.drop(['ProdTaken'],axis=1)
y = df['ProdTaken']
from sklearn.model_selection import train_test_split

y.value_counts()
X.head()
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.20,random_state=42)
X_train.shape
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
num_transform = StandardScaler()
cat_transformer = OneHotEncoder(drop='first')
preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", cat_transformer,categorical_features),
        ("StandardScaler", num_transform, numerical_features)
    ]
)
preprocessor
X_test
categorical_features = [col for col in categorical_features if col in X_train.columns]
numerical_features = [col for col in numerical_features if col in X_train.columns]

X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)
pd.DataFrame(X_train)
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix,precision_score,recall_score,f1_score,roc_auc_score
models={
    'RandomForestClassifier' : RandomForestClassifier(),
    'AdaBoostClassifier': AdaBoostClassifier()
}

for name, model in models.items():
    print("Training Model :", name)
    model.fit(X_train, y_train)


    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    print("\n=== 🧠 Training Metrics ===")
    print("Accuracy       :", accuracy_score(y_train, y_train_pred))
    print("Precision      :", precision_score(y_train, y_train_pred, average='binary'))
    print("Recall         :", recall_score(y_train, y_train_pred, average='binary'))
    print("F1 Score       :", f1_score(y_train, y_train_pred, average='binary'))
    print("ROC AUC Score  :", roc_auc_score(y_train, y_train_pred))
    print("Confusion Matrix:\n", confusion_matrix(y_train, y_train_pred))
    print("Classification Report:\n", classification_report(y_train, y_train_pred))

    print("\n=== 📊 Testing Metrics ===")
    print("Accuracy       :", accuracy_score(y_test, y_test_pred))
    print("Precision      :", precision_score(y_test, y_test_pred, average='binary'))
    print("Recall         :", recall_score(y_test, y_test_pred, average='binary'))
    print("F1 Score       :", f1_score(y_test, y_test_pred, average='binary'))
    print("ROC AUC Score  :", roc_auc_score(y_test, y_test_pred))
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_test_pred))
    print("Classification Report:\n", classification_report(y_test, y_test_pred))
rf ={
    'max_depth':[5,8,None,10],
    'max_features': [5,7,'auto',8],
    'min_samples_split':[2,8,15,20],
    'n_estimators': [100,200,500,1000]
}
ada_boost_params = {
    'n_estimators': [50,60,70,80,90],
    'algorithm': ['SAMME',"SAMME.R"]
}
randomcv_models= [
    ('RF', RandomForestClassifier(),rf),
    ('AdaBoost', AdaBoostClassifier(),ada_boost_params),
]
from sklearn.model_selection import RandomizedSearchCV
model_params = {}
for name , model , params in randomcv_models:
    random = RandomizedSearchCV(estimator=model,param_distributions=params,n_iter=100,cv=3,verbose=2,n_jobs=-1)
    random.fit(X_train,y_train)
    model_params[name] = random.best_params_

for model_name in model_params:
    print(f'-------------Best Params for {model_name}------------')
    print(model_params[model_name])
models={
    'RandomForestClassifier' : RandomForestClassifier(n_estimators= 1000, min_samples_split = 2, max_features = 8, max_depth = None),
    'AdaBoostClassifier' : AdaBoostClassifier(n_estimators= 80, algorithm = 'SAMME'),

}

for name, model in models.items():
        print("Training Model :", name)
        model.fit(X_train, y_train)


        y_train_pred = model.predict(X_train)
        y_test_pred = model.predict(X_test)

        print("\n=== 🧠 Training Metrics ===")
        print("Accuracy       :", accuracy_score(y_train, y_train_pred))
        print("Precision      :", precision_score(y_train, y_train_pred))
        print("Recall         :", recall_score(y_train, y_train_pred))
        print("F1 Score       :", f1_score(y_train, y_train_pred))
        print("ROC AUC Score  :", roc_auc_score(y_train, y_train_pred))
        print("Confusion Matrix:\n", confusion_matrix(y_train, y_train_pred))
        print("Classification Report:\n", classification_report(y_train, y_train_pred))

        print("\n=== 📊 Testing Metrics ===")
        print("Accuracy       :", accuracy_score(y_test, y_test_pred))
        print("Precision      :", precision_score(y_test, y_test_pred))
        print("Recall         :", recall_score(y_test, y_test_pred))
        print("F1 Score       :", f1_score(y_test, y_test_pred))
        print("ROC AUC Score  :", roc_auc_score(y_test, y_test_pred))
        print("Confusion Matrix:\n", confusion_matrix(y_test, y_test_pred))
        print("Classification Report:\n", classification_report(y_test, y_test_pred))





Numerical Features: ['ProdTaken', 'Age', 'CityTier', 'DurationOfPitch', 'NumberOfFollowups', 'PreferredPropertyStar', 'NumberOfTrips', 'Passport', 'PitchSatisfactionScore', 'OwnCar', 'MonthlyIncome', 'Total Visiting']
The Sum of numerical Features is : 12
['Age', 'CityTier', 'DurationOfPitch', 'NumberOfFollowups', 'PreferredPropertyStar', 'NumberOfTrips', 'Passport', 'PitchSatisfactionScore', 'OwnCar', 'MonthlyIncome', 'Total Visiting']
Categorical Features: ['TypeofContact', 'Occupation', 'Gender', 'ProductPitched', 'MaritalStatus', 'Designation']
The Sum of numerical Features is : 6
Discrete Features: ['ProdTaken', 'CityTier', 'Passport', 'PitchSatisfactionScore', 'OwnCar']
Count of Discrete Features: 5
Continuous Features: ['Age', 'DurationOfPitch', 'NumberOfFollowups', 'PreferredPropertyStar', 'NumberOfTrips', 'MonthlyIncome', 'Total Visiting']
Count of Continuous Features: 7
Training Model : RandomForestClassifier

=== 🧠 Training Metrics ===
Accuracy       : 1.0
Precision      : 1

In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier

In [13]:
print(models.values())

dict_values([RandomForestClassifier(max_features=8, n_estimators=1000), AdaBoostClassifier(algorithm='SAMME', n_estimators=80)])
